In [2]:
# Import necessary libraries
import gensim
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
import pandas as pd

In [3]:
class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0
        self.loss_to_be_subed = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        loss_now = loss - self.loss_to_be_subed
        self.loss_to_be_subed = loss
        print('Loss after epoch {}: {}'.format(self.epoch, loss_now))
        self.epoch += 1

In [4]:

df = pd.read_json("reviews_Cell_Phones_and_Accessories_5.json", lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [5]:
review_text = df.reviewText.apply(gensim.utils.simple_preprocess)

In [12]:
display(review_text[23])

['they',
 'are',
 'nothing',
 'special',
 'for',
 'sure',
 'but',
 'it',
 'nice',
 'that',
 'you',
 'can',
 'tell',
 'when',
 'it',
 'is',
 'powered',
 'up',
 'by',
 'the',
 'led',
 'that',
 'glows',
 'in',
 'it',
 'able',
 'to',
 'charge',
 'two',
 'phones',
 'at',
 'once',
 'in',
 'the',
 'car',
 'which',
 'is',
 'nice']

In [17]:
# Now, write the preprocessed text to a .txt file
with open('reviews_processed.txt', 'w') as file:
    for text_list in review_text:
        sentence = ''
        for word in text_list:
            sentence = sentence + ' ' + word
        # # Join the words into a single string with space separation
        # processed_text = '.' + ' ' + (sentence)
        # Write the processed text to a file and add a newline to separate each review
        file.write(sentence + '.\n')

In [6]:
model = Word2Vec(
    window=5,
    min_count=2,
    workers=8,
    compute_loss=True
    )

In [7]:
model.build_vocab(review_text, progress_per=1000)

In [8]:
model.train(review_text, total_examples=model.corpus_count, epochs=3, compute_loss=True, callbacks=[callback()])

Loss after epoch 0: 3301165.75
Loss after epoch 1: 2853682.25
Loss after epoch 2: 2531751.0


(36904813, 50321385)

In [8]:
model.save("./reviews_Cell_Phones_and_Accessories_5.model")

In [11]:
model.wv.most_similar("bad")

[('terrible', 0.6699979305267334),
 ('horrible', 0.6491087079048157),
 ('good', 0.6162761449813843),
 ('shabby', 0.5600782632827759),
 ('obvious', 0.5506466031074524),
 ('cheap', 0.5480924248695374),
 ('ridiculous', 0.5447983741760254),
 ('awful', 0.5359783172607422),
 ('weird', 0.5234681367874146),
 ('okay', 0.5146394371986389)]